#Import Models

In [1]:
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=1c1cede0260b91301eec0761660cccce75e2ae8b14cf68e23c24b81029b2c889
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd

In [3]:
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

#Connect to sharepoint

In [4]:
!pip install office365
!pip install Office365-REST-Python-Client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.files.file import File

####inputs########
# This will be the URL that points to your sharepoint site.
# Make sure you change only the parts of the link that start with "Your"
url_shrpt = 'https://ihuedu.sharepoint.com/sites/EDYTEProject2023/'
username_shrpt = '###############'
password_shrpt = '###############'
folder_files_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/keywords/keywords_topics/'
folder_analysis_url_shrpt = '/sites/EDYTEProject2023/Shared%20Documents/General/wp5_data/analysis_exports/'

In [6]:
###Authentication###For authenticating into your sharepoint site###
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
  ctx = ClientContext(url_shrpt, ctx_auth)
  web = ctx.web
  ctx.load(web)
  ctx.execute_query()
  print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
  print(ctx_auth.get_last_error())
############################

Authenticated into sharepoint as:  EDYTE Project 2023


In [7]:
####Function for extracting the file names of a folder in sharepoint###
###If you want to extract the folder names instead of file names, you have to change "sub_folders = folder.files" to "sub_folders = folder.folders" in the below function

global print_folder_contents
def print_folder_contents(ctx, folder_url):
    try:

        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        fold_names = []
        sub_folders = folder.files #Replace files with folders for getting list of folders
        ctx.load(sub_folders)
        ctx.execute_query()

        for s_folder in sub_folders:

            fold_names.append(s_folder.properties["Name"])

        return fold_names

    except Exception as e:
        print('Problem printing out library contents: ', e)
######################################################

# Call the function by giving your folder URL as input
filelist_shrpt=print_folder_contents(ctx,folder_files_url_shrpt)
#Print the list of files present in the folder
print(filelist_shrpt)

['process_evidences_keywords_topic.csv', 'steps_digital_keywords_topic.csv', 'process_keywords_topic.csv', 'process_conditions_keywords_topic.csv', 'process_rules_keywords_topic.csv', 'steps_keywords_topic.csv']


# Import Data

In [8]:
import pandas as pd
import csv
import numpy as np
import io
import os
import tempfile

In [9]:
def process_csv_file(file_name):
    file_url= folder_files_url_shrpt+file_name
    response = File.open_binary(ctx, file_url)  # Assuming File is imported and ctx is defined
    df = pd.read_csv(io.BytesIO(response.content))
    return df

In [10]:
#steps
steps_keywords_topic = process_csv_file("steps_keywords_topic.csv")

#steps digital
steps_digital_keywords_topic  = process_csv_file( "steps_digital_keywords_topic.csv")

#process
process_keywords_topic = process_csv_file( "process_keywords_topic.csv")

#process evidences
process_evidences_keywords_topic= process_csv_file( "process_evidences_keywords_topic.csv")

#process conditions
process_conditions_keywords_topic = process_csv_file( "process_conditions_keywords_topic.csv")

#process rules
process_rules_keywords_topic = process_csv_file( "process_rules_keywords_topic.csv")

In [11]:
process_rules_keywords_topic

,process_id,ihu_unique_rule_id,keywords,topic
0,439993,439993_rul_01,170 αποδεικτικό ενημερότητας,66
1,439993,439993_rul_01,2013,155
2,439993,439993_rul_01,τροποποιήθηκε άρθρο 51,5
3,439993,439993_rul_02,1265 2015,1
4,439993,439993_rul_02,απόφαση γενικού,58
...,...,...,...,...
34684,997316,997316_rul_01,μητρώο αδειών,171
34685,997316,997316_rul_01,αυτοκινήτων on line,265
34686,997316,997316_rul_02,αυτοκινήτων on line,265
34687,997316,997316_rul_02,μητρώο αδειών,171


# Analysis on Entities

## Functions

In [12]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

def get_sentence_embeddings(sentences, model):
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True, batch_size=32)
    return sentence_embeddings.cpu()

def build_faiss_index(embeddings):
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    return index

def find_most_similar_sentence(query_embedding, index, embeddings, similarity_threshold, kept_sentences):
    similarity_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_similarities = similarity_scores[similarity_scores >= similarity_threshold]
    if len(top_similarities) > 0:
        max_similarity = top_similarities.max().item()
        max_similarity_index = np.argmax(similarity_scores)
        if max_similarity >= similarity_threshold:
            return kept_sentences[max_similarity_index], max_similarity
    return '', 0.0

def filter_similar_sentences(dataframe, model, similarity_threshold=0.95):
    # Create new columns to store the most similar sentence and similarity measure
    dataframe['final_keywords'] = ''
    dataframe['similarity_measure'] = 0.0

    # Initialize the kept_sentences list and embeddings
    kept_sentences = []
    kept_embeddings = np.empty((0, model.get_sentence_embedding_dimension()), dtype=np.float32)
    kept_index = None

    # Iterate over the rows in the dataframe with progress bar
    for i, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Processing"):
        curr_sentence = row['keywords']
        if isinstance(curr_sentence, str):  # Check if the sentence is a valid string
            curr_embedding = get_sentence_embeddings([curr_sentence], model)[0]  # Pass the model as an argument
            most_similar_sentence = ''
            max_similarity = 0.0

            # Compare with the previously kept sentences using FAISS index
            if kept_index is not None:
                most_similar_sentence, max_similarity = find_most_similar_sentence(
                    curr_embedding, kept_index, kept_embeddings, similarity_threshold, kept_sentences
                )

            # If a similar sentence is found, update the most similar sentence and similarity measure columns
            if most_similar_sentence != '':
                dataframe.loc[i, 'final_keywords'] = most_similar_sentence
                dataframe.loc[i, 'similarity_measure'] = max_similarity
            else:
                # If no similar sentence is found, keep the original text instead of the embedding
                kept_sentences.append(curr_sentence)
                dataframe.loc[i, 'final_keywords'] = curr_sentence
                dataframe.loc[i, 'similarity_measure'] = 1.0  # Set similarity measure to 1.0 for the original sentence

            # Check if the current sentence is similar to the kept sentences
            if most_similar_sentence == '' or max_similarity < similarity_threshold:
                kept_embeddings = np.vstack((kept_embeddings, curr_embedding.cpu().numpy()))
                if kept_index is None:
                    kept_index = build_faiss_index(np.array([curr_embedding.cpu().numpy()]))
                else:
                    kept_embeddings_array = np.array(kept_embeddings)
                    new_embeddings_array = np.array([curr_embedding.cpu().numpy()])
                    combined_embeddings = np.vstack((kept_embeddings_array, new_embeddings_array))
                    kept_index.add(combined_embeddings)

        else:
            dataframe.loc[i, 'final_keywords'] = np.nan
            dataframe.loc[i, 'similarity_measure'] = np.nan

    return dataframe


In [13]:
def process_input_data(input_data):

    topics_num = input_data["topic"].nunique() - 1
    model_name = 'paraphrase-multilingual-mpnet-base-v2'
    model = SentenceTransformer(model_name)

    homogenised_keywords = pd.DataFrame()

    for j in range(-1, topics_num):
        sentences = input_data.loc[input_data['topic'] == j]
        filtered_df = filter_similar_sentences(sentences, model, similarity_threshold=0.975)

        homogenised_keywords = pd.concat([homogenised_keywords, filtered_df], ignore_index=True)
        # print(j, final_steps)

    return homogenised_keywords

In [14]:
!python -m spacy download el_core_news_lg

2023-07-20 10:20:46.160023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.6/568.6 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_lg')


In [15]:
import spacy
nlp = spacy.load("el_core_news_lg")

def lemmatize_keywords(row):
    doc = nlp(row['final_keywords'])
    lemmas = " ".join([token.lemma_ for token in doc])
    return lemmas

In [16]:
def extract_entity_ids(df):
    # Extract unique keywords
    keywords = set(df['final_keywords'])

    # Assign IDs to keywords
    keyword_ids = {keyword: i for i, keyword in enumerate(keywords, start=1)}

    # Create the dictionary of unique keywords and IDs
    keyword_dict = {keyword: keyword_ids[keyword] for keyword in keywords}

    return keyword_dict

In [17]:
def add_entity_ids(df, dictionary):

    # Add the "Entity ID" column to the dataframe
    df['keyword_id'] = df['final_keywords'].apply(lambda x: dictionary[x])

    return df

In [18]:
def linked_keywords(df, dictionary):
   df['Linked Keywords'] = df['final_keywords'].apply( lambda x: ', '.join(str(dictionary[word]) for word in x.split() if word in dictionary) if len(x.split()) > 1 else '')

   return df

# Run the process

## Similarity & Lemmatization

### Steps

In [67]:
h_steps_keywords = process_input_data(steps_keywords_topic)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 74/74 [00:01<00:00, 66.82it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [68]:
# Apply the lemmatization function to each row
h_steps_keywords['final_keywords'] = h_steps_keywords.apply(lemmatize_keywords, axis=1)

### Steps Digital

In [19]:
h_steps_digital_keywords = process_input_data(steps_digital_keywords_topic)

<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 402/402 [00:16<00:00, 23.96it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [20]:
# Apply the lemmatization function to each row
h_steps_digital_keywords['final_keywords'] = h_steps_digital_keywords.apply(lemmatize_keywords, axis=1)

### Process

In [21]:
h_process_keywords = process_input_data(process_keywords_topic)

<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 1282/1282 [00:23<00:00, 55.72it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [22]:
# Apply the lemmatization function to each row
h_process_keywords['final_keywords'] = h_process_keywords.apply(lemmatize_keywords, axis=1)

### Process Evidences

In [69]:
h_process_evidences_keywords = process_input_data(process_evidences_keywords_topic)

Streaming output truncated to the last 5000 lines.
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 187/187 [00:03<00:00, 61.55it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [70]:
# Apply the lemmatization function to each row
h_process_evidences_keywords['final_keywords'] = h_process_evidences_keywords.apply(lemmatize_keywords, axis=1)

### Process Conditions

In [71]:
h_process_conditions_keywords = process_input_data(process_conditions_keywords_topic)

<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 3804/3804 [01:24<00:00, 45.09it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [72]:
# Apply the lemmatization function to each row
h_process_conditions_keywords['final_keywords'] = h_process_conditions_keywords.apply(lemmatize_keywords, axis=1)

### Process Rules

In [73]:
h_process_rules_keywords = process_input_data(process_rules_keywords_topic)

<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['final_keywords'] = ''
<ipython-input-12-bbd72a401d1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['similarity_measure'] = 0.0
Processing: 100%|██████████| 2712/2712 [00:50<00:00, 54.10it/s]
<ipython-input-12-bbd72a401d1f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [74]:
# Apply the lemmatization function to each row
h_process_rules_keywords['final_keywords'] = h_process_rules_keywords.apply(lemmatize_keywords, axis=1)

## Dictionary

In [75]:
merged_df = pd.concat([h_steps_keywords['final_keywords'], h_steps_digital_keywords['final_keywords'], h_process_keywords['final_keywords'], h_process_evidences_keywords['final_keywords'], h_process_conditions_keywords['final_keywords'], h_process_rules_keywords['final_keywords']],  ignore_index=True)
merged_df = pd.DataFrame (merged_df)

In [76]:
keywords_dict = extract_entity_ids(merged_df)

## Add Entity ID and Linked Keywords

### Steps

In [82]:
# Assign ID
steps_keywords = add_entity_ids(h_steps_keywords, keywords_dict)

In [83]:
# Linked Keywords
steps_keywords = linked_keywords(steps_keywords, keywords_dict)

In [84]:
steps_keywords

,process_id,ihu_unique_step_id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,616541,616541_stp_06,ιι τύπου 12,-1,ιι τύπος 12,1.0,38457,"47645, 35392, 46849"
1,616541,616541_stp_10,σιωπηρή,-1,σιωπηρή,1.0,33009,
2,616541,616541_stp_12,ιι τύπου 12,-1,ιι τύπος 12,1.0,38457,"47645, 35392, 46849"
3,374859,374859_stp_03,βεβαίωσης ανεργίας,-1,βεβαίωση ανεργία,1.0,45995,"4323, 43669"
4,161114,161114_stp_06,αποθήκευση,-1,αποθήκευση,1.0,25362,
...,...,...,...,...,...,...,...,...
68625,787114,787114_stp_05,οριστικοποίηση,590,οριστικοποίηση,1.0,44572,
68626,393191,393191_stp_03,οριστικοποίηση,590,οριστικοποίηση,1.0,44572,
68627,782134,782134_stp_03,οριστικοποίηση,590,οριστικοποίηση,1.0,44572,
68628,414891,414891_stp_06,οριστικοποίηση,590,οριστικοποίηση,1.0,44572,


### Steps Digital

In [85]:
# Assign ID
steps_digital_keywords = add_entity_ids(h_steps_digital_keywords, keywords_dict)

In [86]:
# Linked Keywords
steps_digital_keywords = linked_keywords(steps_digital_keywords, keywords_dict)

In [87]:
steps_digital_keywords

,process_id,ihu_unique_step_digital_id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,439605,439605_stdg_06,ανταποκριτή ελγα,-1,ανταποκριτής ελγα,1.0,20,"20115, 45314"
1,872303,872303_stdg_01,δυπα πρώην,-1,δυπα πρώην,1.0,43621,"18022, 26692"
2,872303,872303_stdg_02,δυπα πρώην,-1,δυπα πρώην,1.0,43621,"18022, 26692"
3,872303,872303_stdg_03,ααδε,-1,ααδε,1.0,13426,
4,872303,872303_stdg_07,ανανέωση δελτίου,-1,ανανέωση δελτίου,1.0,2057,"24686, 46125"
...,...,...,...,...,...,...,...,...
4980,264600,264600_stdg_01,ymς,208,ymς,1.0,42116,
4981,554454,554454_stdg_15,γγπσδδ,208,γγπσδδ,1.0,23300,
4982,596897,596897_stdg_01,ymς,208,ymς,1.0,42116,
4983,524651,524651_stdg_03,επαλ,208,επαλ,1.0,2229,


### Process

In [88]:
# Assign ID
process_keywords = add_entity_ids(h_process_keywords,  keywords_dict)

In [89]:
# Linked Keywords
process_keywords = linked_keywords(process_keywords, keywords_dict)

In [90]:
process_keywords

,id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,937099,διακίνησης,-1,διακίνηση,1.000000,13161,
1,616541,iι,-1,iι,1.000000,24188,
2,820937,απογραφής,-1,απογραφή,1.000000,47774,
3,977354,ποινικού μητρώου,-1,ποινικός μητρώου,1.000000,47996,8969
4,161114,αυτοψία,-1,αυτοψία,1.000000,9957,
...,...,...,...,...,...,...,...
9055,305551,προσωρινή άδεια,361,προσωρινός άδεια,0.997540,12032,"6570, 7653"
9056,926622,άδεια προσωρινής,361,προσωρινός άδεια,0.988419,12032,"6570, 7653"
9057,102975,υπαγωγή καθεστώς προσωρινής,361,υπαγωγή καθεστώς προσωρινός,1.000000,23686,"24775, 8389, 6570"
9058,531754,προσωρινή παραχώρηση χρήσης,361,προσωρινός παραχώρηση χρήση,1.000000,35303,"6570, 28395, 29475"


### Process Evidences

In [93]:
# Assign ID
process_evidences_keywords = add_entity_ids(h_process_evidences_keywords, keywords_dict)


In [94]:
# Linked Keywords
process_evidences_keywords = linked_keywords(process_evidences_keywords, keywords_dict)

In [95]:
process_evidences_keywords

,process_id,ihu_unique_evidence_id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,616541,616541_ev_03,έκτακτης ανάγκης,-1,έκτακτος ανάγκη,1.000000,42985,"13012, 2481"
1,963466,963466_ev_09,είναι χρήστης ναρκωτικών,-1,είμαι χρήστης ναρκωτικός,1.000000,49455,"19632, 41620, 25304"
2,214376,214376_ev_08,1976 όπως ισχύει,-1,1976 όπως ισχύω,1.000000,7914,"34232, 25904, 5940"
3,214376,214376_ev_14,διατροφής θανόντα,-1,διατροφή θανόντας,1.000000,1139,"49363, 33175"
4,814359,814359_ev_11,supplement,-1,supplement,1.000000,9491,
...,...,...,...,...,...,...,...,...
41558,751895,751895_ev_03,διοίκησης νόμιμης εκπροσώπησης,417,διοίκηση νόμιμος εκπροσώπηση,1.000000,46475,"7737, 8431, 36603"
41559,218647,218647_ev_03,διοίκησης νόμιμης εκπροσώπησης,417,διοίκηση νόμιμος εκπροσώπηση,1.000000,46475,"7737, 8431, 36603"
41560,196487,196487_ev_03,διοίκησης νόμιμης εκπροσώπησης,417,διοίκηση νόμιμος εκπροσώπηση,1.000000,46475,"7737, 8431, 36603"
41561,952966,952966_ev_01,νόμιμος εκπρόσωπος υπό,417,νόμιμος εκπρόσωπος υπό,1.000000,16890,"8431, 12509, 5068"


### Process Conditions

In [109]:
# Assign ID
process_conditions_keywords = add_entity_ids(h_process_conditions_keywords, keywords_dict)

In [110]:
# Linked Keywords
process_conditions_keywords = linked_keywords(process_conditions_keywords, keywords_dict)

In [111]:
process_conditions_keywords

,process_id,ihu_unique_condition_id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,937099,937099_con_06,προέρχονται ίδια,-1,προέρχω ίδιος,1.000000,19358,32024
1,616541,616541_con_04,εκπαιδευτεί επαρκώς ώστε,-1,εκπαιδευτεί επαρκώς ώστε,1.000000,33129,2237
2,616541,616541_con_05,ψηφιακές υπηρεσίες υπουργείου,-1,ψηφιακός υπηρεσία υπουργείο,1.000000,4007,"33061, 14993, 27815"
3,963466,963466_con_04,είναι χρήστης ναρκωτικών,-1,είμαι χρήστης ναρκωτικός,1.000000,49455,"19632, 41620, 25304"
4,439605,439605_con_01,ως ανταποκριτής,-1,ως ανταποκριτής,1.000000,48941,"49992, 20115"
...,...,...,...,...,...,...,...,...
31529,592187,592187_con_02,μεταφορών,252,μεταφορικός,0.986728,36361,
31530,157554,157554_con_03,ταχυμεταφορά courier,252,ταχυμεταφορά courier,1.000000,16375,
31531,749807,749807_con_07,μισθωμένα μεταφορικά μέσα,252,μισθωμένος μεταφορικός μέσο,1.000000,48481,"36361, 2469"
31532,624363,624363_con_02,μεταφορές,252,μεταφορικός,0.983351,36361,


### Process Rules

In [96]:
# Assign ID
process_rules_keywords = add_entity_ids(h_process_rules_keywords, keywords_dict)

In [97]:
# Linked Keywords
process_rules_keywords = linked_keywords(process_rules_keywords, keywords_dict)

In [98]:
process_rules_keywords

,process_id,ihu_unique_rule_id,keywords,topic,final_keywords,similarity_measure,keyword_id,Linked Keywords
0,439993,439993_rul_04,1124,-1,1124,1.0,34219,
1,439993,439993_rul_06,εσόδων 1002,-1,εσόδων 1002,1.0,30415,158
2,439993,439993_rul_07,ανεξάρτητης,-1,ανεξάρτητος,1.0,4830,
3,523764,523764_rul_01,στρατηγικής 2012 2015,-1,στρατηγική 2012 2015,1.0,3992,"19789, 25209, 45527"
4,937099,937099_rul_04,συμμόρφωση οδηγίες 64,-1,συμμόρφωση οδηγία 64,1.0,49875,"30656, 32339"
...,...,...,...,...,...,...,...,...
34684,137848,137848_rul_02,κυκλοφορίας μικτής χρήσης,321,κυκλοφορία μικτός χρήση,1.0,1184,"10902, 29475"
34685,822484,822484_rul_01,διεκπεραιώνονται μέσω κεπ,321,διεκπεραιώνομαι μέσω κεπ,1.0,576,"24741, 756"
34686,987972,987972_rul_01,διεκπεραιώνονται μέσω κεπ,321,διεκπεραιώνομαι μέσω κεπ,1.0,576,"24741, 756"
34687,670595,670595_rul_01,διεκπεραιώνονται μέσω κεπ,321,διεκπεραιώνομαι μέσω κεπ,1.0,576,"24741, 756"


# Save Data

In [103]:
# load csv module
import csv

def upload_csv_to_target_folder(name, dataframe):

  path = name+".csv"

  dataframe.to_csv(path, index=False)

  url=folder_analysis_url_shrpt+"keywords"
  target_folder = ctx.web.get_folder_by_server_relative_url(url)

  with open(path, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path), file_content).execute_query()

In [100]:
# load csv module
import csv

def upload_dic_to_target_folder(name, dict):

  path2 = name+".csv"

  # open file for writing, "w" is writing
  w = csv.writer(open(path2, "w"))

  # loop over dictionary keys and values
  for key, val in dict.items():

      # write every key and value to file
      w.writerow([key, val])

  url=folder_analysis_url_shrpt+"keywords"
  target_folder = ctx.web.get_folder_by_server_relative_url(url)

  with open(path2, "rb") as content_file:
      file_content = content_file.read()
      target_folder.upload_file(os.path.basename(path2), file_content).execute_query()

In [104]:
upload_csv_to_target_folder("steps_keywords", steps_keywords)

In [105]:
upload_csv_to_target_folder("steps_digital_keywords", steps_digital_keywords)

In [106]:
upload_csv_to_target_folder("process_keywords", process_keywords)

In [107]:
upload_csv_to_target_folder("process_evidences_keywords", process_evidences_keywords)

In [112]:
upload_csv_to_target_folder("process_conditions_keywords", process_conditions_keywords)

In [113]:
upload_csv_to_target_folder("process_rules_keywords", process_rules_keywords)

In [114]:
upload_dic_to_target_folder("keywords_dic", keywords_dict)